# Biostat 257 Homework 4

**Due May 28 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW2
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T,
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite, we use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW5, we will derive a EM (expectation-maximization) algorithm for the same problem. 

In [2]:
# load necessary packages; make sure to install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathProgBase, MixedModels, NLopt
using Random, RCall, Revise

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Show that the gradient of $\ell_i$ is
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{eqnarray*}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.

### Part 1

For $\nabla_{\boldsymbol{\beta}} \ell_i$,
\begin{eqnarray*}
\text{d} \ell_i &=& \frac{1}{2} (\mathbf{X}_i \text{d} \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}) + \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i \text{d} \boldsymbol{\beta}\\
&=& (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i \text{d} \boldsymbol{\beta} = \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i \text{d} \boldsymbol{\beta} \\
\therefore \text{D}_{\boldsymbol{\beta}} \ell_i &=& \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i  \\
\nabla_{\boldsymbol{\beta}} \ell_i &=& (\text{D}_{\boldsymbol{\beta}} \ell_i)^T =\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i
\end{eqnarray*}

For $\nabla_{\sigma^2} \ell_i$,
\begin{eqnarray*}
\text{d} \ell_i &=& - \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-1} \text{d}\boldsymbol{\Omega}_i) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \text{d}\boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \\
&=& - \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-1} \text{d}\sigma^2) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i \text{d}\sigma^2 \quad (\because \text{d}\boldsymbol{\Omega}_i = \text{d} \sigma^2 \mathbf I) \\
\therefore \text{D}_{\sigma^2} \ell_i &=& - \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i = \nabla_{\sigma^2} \ell_i \\
\end{eqnarray*}

For $\frac{\partial}{\partial \mathbf{L}} \ell_i$,
\begin{eqnarray*}
\text{d} \ell_i &=& - \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-1} \text{d}\boldsymbol{\Omega}_i) + \frac{1}{2} \text{tr} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \text{d}\boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i) \\
\text{d} \boldsymbol{\Omega}_i &=& \mathbf{Z}_i \text{d} \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T + \mathbf{Z}_i \mathbf{L} \text{d} \mathbf{L}^T \mathbf{Z}_i^T
\end{eqnarray*}

By substituting and rearranging terms inside trace, it is straightforward to show that
\begin{eqnarray*}
\text{d} \ell_i &=& \text{tr} (- \mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \text{d} \mathbf{L} + \mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \text{d} \mathbf{L}) \\
\therefore \frac{\partial}{\partial \mathbf{L}} \ell_i &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}
\end{eqnarray*}

For $\text{D}_{\text{vech} \mathbf{L}} \ell_i$, it can be also shown that
\begin{eqnarray*}
\text{d} \ell_i &=& \text{vec} (- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \text{dvec} \mathbf{L} \\
&=& \text{vec} (- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \mathbf{D}_q \text{dvech} \mathbf{L}
\end{eqnarray*}
where $\mathbf{D}_q$ is the triangular duplication matrix such that
\begin{eqnarray*}
\text{D}_{\text{vech} \mathbf{L}} \ell_i &=& \text{vec} (- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \mathbf{D}_q.
\end{eqnarray*}
Alternatively,  
\begin{eqnarray*}
\text{d} \ell_i &=& \text{vec} (- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \text{dvec} \mathbf{L} + \frac{1}{2}\text{vec} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \text{d} \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i) + \frac{1}{2}\text{vec} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \text{d} \mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i) \\
&=& \text{vec} (- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \text{dvec} \mathbf{L} + \frac{1}{2} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \text{dvec} \mathbf{L} + \frac{1}{2} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \text{dvec} \mathbf{L}^T \\
&=& \text{vec} (- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \text{dvec} \mathbf{L} + \frac{1}{2} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \text{dvec} \mathbf{L} + \frac{1}{2} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq}\text{dvec} \mathbf{L}
\end{eqnarray*}
where $\mathbf{K}_{qq}$ is the commutation matrix such that
\begin{eqnarray*}
\text{D}_{\text{vech} \mathbf{L}} \ell_i &=& \text{vec} (- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \mathbf{D}_q + \frac{1}{2} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \mathbf{D}_q + \frac{1}{2} (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \mathbf{D}_q.
\end{eqnarray*}
The above expressions for $\text{D}_{\text{vech} \mathbf{L}} \ell_i$ are equivalent, since $\text{vec} (\boldsymbol{a}\boldsymbol{b}^T) = \boldsymbol{b} \otimes \boldsymbol{a}$.

### Part 2

For observed and expected Fisher information matrices,
\begin{eqnarray*}
\text{d} (\nabla_{\boldsymbol{\beta}} \ell_i) &=& -\mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i \text{d} \boldsymbol{\beta} \\
\therefore - \nabla^2_{\boldsymbol{\beta}} \ell_i &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i \rightarrow \mathbb{E}[- \nabla^2_{\boldsymbol{\beta}} \ell_i] = \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{X}_i
\end{eqnarray*}

\begin{eqnarray*}
\text{d}(\frac{\partial}{\partial \boldsymbol{\beta}} \ell_i) &=& - \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i = - \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i \text{d} \sigma^2\\
\therefore \frac{\partial^2}{\partial \sigma^2 \partial \boldsymbol{\beta}} \ell_i &=&  - \mathbf{X}_i^T \boldsymbol{ \Omega}_i^{-2} \mathbf{r}_i \rightarrow \mathbb{E}[- \frac{\partial^2}{\partial \sigma^2 \partial \boldsymbol{\beta}} \ell_i] = \mathbf{0}_{p \times 1}
\end{eqnarray*}

\begin{eqnarray*}
\text{d}(\frac{\partial}{\partial \boldsymbol{\beta}} \ell_i) &=& - \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \\
&=& - \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} (\mathbf{Z}_i \text{d} \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T + \mathbf{Z}_i \mathbf{L} \text{d} \mathbf{L}^T \mathbf{Z}_i^T) \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \\
&=& - (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i ) \text{dvec} \mathbf{L} - (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \text{dvec} \mathbf{L}^T  \\
&=& - (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i ) \text{dvec} \mathbf{L} - (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq}\text{dvec} \mathbf{L} \\
\therefore \frac{\partial^2}{\partial (\text{vech} \mathbf{L})^T \partial \boldsymbol{\beta}} \ell_i &=&  - (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i ) \mathbf{D}_q - (\mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \mathbf{D}_q \\ 
&\rightarrow& \mathbb{E}[- \frac{\partial^2}{\partial (\text{vech} \mathbf{L})^T \partial \boldsymbol{\beta}} \ell_i] = \mathbf{0}_{p \times \frac{q(q+1)}{2}}
\end{eqnarray*}

\begin{eqnarray*}
\text{d} (\nabla_{\sigma^2} \ell_i) &=& \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1}) - \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i - \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i  \\
&=& \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-2} \text{d} \sigma^2) - \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-3} \mathbf{r}_i \text{d} \sigma^2 \quad (\because \text{d}\boldsymbol{\Omega}_i = \text{d} \sigma^2 \mathbf I) \\
\therefore \nabla^2_{\boldsymbol{\sigma}^2} \ell_i &=& \frac{\partial^2}{(\partial \sigma^2)^2} \ell_i = \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-2}) - \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-3} \mathbf{r}_i \rightarrow \mathbb{E} [- \nabla^2_{\boldsymbol{\sigma}^2} \ell_i] = - \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-2}) + \text{tr} (\boldsymbol{\Omega}_i^{-2}) = \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-2})
\end{eqnarray*}

\begin{eqnarray*}
\text{d} (\nabla_{\sigma^2} \ell_i) &=& \frac{1}{2} \text{tr} (\boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1}) - \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i - \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i  \\
&=& \text{tr} (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{Z}_i \text{d} \mathbf{L} - \mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \text{d} \mathbf{L} - \mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{Z}_i \text{d} \mathbf{L}) \\
&=& \text{vec} (\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{Z}_i \mathbf{L} - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{Z}_i \mathbf{L} - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \mathbf{D}_q \text{dvech} \mathbf{L} \\
\therefore \frac{\partial^2}{\partial (\text{vech} \mathbf{L})^T \partial \sigma^2} \ell_i &=& \text{vec} (\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{Z}_i \mathbf{L} - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{Z}_i \mathbf{L} - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L})^T \mathbf{D}_q \\ 
&\rightarrow& \mathbb{E}[- \frac{\partial^2}{\partial (\text{vech} \mathbf{L})^T \partial \sigma^2} \ell_i] = \text{vec}(\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{Z}_i \mathbf{L})^T \mathbf{D}_q \quad (\because \mathbb{E}[\mathbf{r}_i \mathbf{r}_i^T] = \boldsymbol{\Omega}_i)
\end{eqnarray*}

\begin{eqnarray*}
\text{d} (\text{D}_{\text{vech} \mathbf{L}} \ell_i)^T &=& \mathbf{D}_q^T \text{vec} (\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \text{d} \mathbf{L} \\
&& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \text{d} \boldsymbol{\Omega}_i \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \text{d} \mathbf{L}) \\
&=& \mathbf{D}_q^T [(\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \text{dvec} \mathbf{L} + (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \text{dvec} \mathbf{L} \\
&& - (\mathbf{I}_q \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \text{dvec} \mathbf{L} - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \text{dvec} \mathbf{L} \\
&& - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \text{dvec} \mathbf{L} \\
&& - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \text{dvec} \mathbf{L} \\
&& - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \text{dvec} \mathbf{L} \\
&& + (\mathbf{I}_q \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \text{dvec} \mathbf{L}] \\
\therefore \frac{\partial^2}{\partial (\text{vech} \mathbf{L})^T \partial \text{vech} \mathbf{L}} \ell_i &=& \mathbf{D}_q^T [(\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) + (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \\
&& - (\mathbf{I}_q \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \\
&& - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \\
&& - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i) \\
&& - (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \\
&& + (\mathbf{I}_q \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i)] \mathbf{D}_q \\
\rightarrow \mathbb{E}[- \frac{\partial^2}{\partial (\text{vech} \mathbf{L})^T \partial \text{vech} \mathbf{L}} \ell_i] &=& \mathbf{D}_q^T [\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i + (\mathbf{L}^T \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \otimes \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq}] \mathbf{D}_q 
\end{eqnarray*}

## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW2 to evaluate both objective and gradient. I provide my code for HW2 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 

In [3]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇L         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    LM⁻¹LᵗZᵗZ     :: Matrix{T}
    ZᵗZLM⁻¹LᵗZᵗr  :: Vector{T}
    Zᵗr           :: Vector{T}
    ZᵗΩ⁻¹r        :: Vector{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇L         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    LM⁻¹LᵗZᵗZ     = Matrix{T}(undef, q, q)
    ZᵗZLM⁻¹LᵗZᵗr  = Vector{T}(undef, q)
    Zᵗr           = Vector{T}(undef, q)
    ZᵗΩ⁻¹r        = Vector{T}(undef, q)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇L, 
        yty, xty, zty, storage_p, storage_q, 
        xtx, ztx, ztz, storage_qq,
        LM⁻¹LᵗZᵗZ, ZᵗZLM⁻¹LᵗZᵗr, Zᵗr, ZᵗΩ⁻¹r)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad == true`, then `obs.∇β`, `obs.∇L`, and `obs.σ²` are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lᵗ Zᵗ Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lᵗ Zᵗ Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.Zᵗr, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.storage_q, obs.Zᵗr)) # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇σ², ∇L by gradients
        # compute ∇β
        copy!(obs.∇β, obs.xty) 
        BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), obs.∇β)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q) # LM⁻¹LᵗZᵗr
        BLAS.gemv!('T', T(-1), obs.ztx, obs.storage_q, T(1), obs.∇β)
        obs.∇β ./= σ²
        # compute ∇σ² 
        obs.∇σ²[1] = - n 
        BLAS.gemm!('T', 'N', T(1), L, obs.ztz, T(0), obs.LM⁻¹LᵗZᵗZ)
        BLAS.trsm!('L', 'U', 'T', 'N', T(1), obs.storage_qq, obs.LM⁻¹LᵗZᵗZ)
        BLAS.trsm!('L', 'U', 'N', 'N', T(1), obs.storage_qq, obs.LM⁻¹LᵗZᵗZ)
        BLAS.trmm!('L', 'L', 'N', 'N', T(1), L, obs.LM⁻¹LᵗZᵗZ)
        obs.∇σ²[1] += tr(obs.LM⁻¹LᵗZᵗZ)
        BLAS.gemv!('N', T(1), obs.ztz, obs.storage_q, T(0), obs.ZᵗZLM⁻¹LᵗZᵗr)
        obs.∇σ²[1] += (rtr - 2 * qf + dot(obs.storage_q, obs.ZᵗZLM⁻¹LᵗZᵗr)) / σ²
        obs.∇σ²[1] /= (2 * σ²)
        # compute ∇L
        copy!(obs.∇L, obs.ztz)
        BLAS.gemm!('N', 'N', T(1), obs.ztz, obs.LM⁻¹LᵗZᵗZ, T(-1), obs.∇L)
        obs.∇L ./= σ²
        obs.ZᵗΩ⁻¹r .= (obs.Zᵗr .- obs.ZᵗZLM⁻¹LᵗZᵗr) ./ σ²
        BLAS.ger!(T(1), obs.ZᵗΩ⁻¹r, obs.ZᵗΩ⁻¹r, obs.∇L)
        # L is multiplied later to reduce flops
    end
    ###################
    # Return
    ###################        
    return logl    
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [4]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [5]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇L;

logl = logl!(obs, β, L, σ², true) = -3261.9177559187597
obs.∇β = [-1.1937579038479573, -20.06810413006247, -12.323942329522415, -5.174641191562259, 28.20847569099026]
obs.∇σ² = [5.50162054089166]
obs.∇L = [0.4073285664341204 -0.10370457194287118 0.9214023976865059; -0.10370457194294816 -0.9907411137746662 -0.02165831118499488; 0.9214023976864656 -0.02165831118503278 -0.5355581080218288]


You will lose all 20 points if following statement throws `AssertionError`.

In [6]:
@assert abs(logl - (-3261.9177559187597)) < 1e-8
@assert norm(obs.∇β - [-1.1937579038479573, -20.068104130062466, 
        -12.323942329522424, -5.174641191562253, 28.208475690990248]) < 1e-8
@assert norm(obs.∇L - [0.40732856643442306 -0.10370457194285436 0.9214023976868783; 
        -0.10370457194285436 -0.9907411137746756 -0.02165831118509598; 
        0.9214023976868783 -0.02165831118509598 -0.5355581080215592]) < 1e-8
@assert abs(obs.∇σ²[1] - (5.501620540891395)) < 1e-8

### Efficiency

Benchmark for evaluating objective only. This is what we did in HW2.

In [7]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     617.486 ns (0.00% GC)
  median time:      663.659 ns (0.00% GC)
  mean time:        718.516 ns (0.00% GC)
  maximum time:     2.280 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     173

Benchmark for objective + gradient evaluation.

In [8]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.794 μs (0.00% GC)
  median time:      1.917 μs (0.00% GC)
  mean time:        2.259 μs (0.00% GC)
  maximum time:     17.449 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

My median runt time is 2.1μs. You will get full credit (10 pts) if the median run time is within 10μs.

In [9]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

In [10]:
# # check for type stability
# @code_warntype logl!(obs, β, L, σ², true)

In [11]:
# using Profile
# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, L, σ², true); end
# Profile.print(format=:flat)

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

In [12]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MathProgBase.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
    storage_pp  :: Matrix{T}
    storage_qq2 :: Matrix{T}
    ztr         :: Vector{T}
    storage_qq  :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    storage_pp = Matrix{T}(undef, p, p)
    storage_qq2 = Matrix{T}(undef, abs2(q), abs2(q))
    ztr         = Vector{T}(undef, q)
    storage_qq  = Matrix{T}(undef, q, q)
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr2, xtx, ztz2,
        storage_pp, storage_qq2, ztr, storage_qq)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad == true`, then `m.∇β`, `m.∇L`, and `m.σ²` are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇L, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    # obtain gradient wrt L: m.∇L = m.∇L * L
    if needgrad
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), m.L, m.∇L)
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a fake longitudinal data set to test our algorithm.

In [13]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.csv`. **Do not put this file in Git.** It takes 246.6MB storage.

In [14]:
(isfile("lmm_data.csv") && filesize("lmm_data.csv") == 244697920) || 
open("lmm_data.csv", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p - 1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q - 1)
        print(io, "Z" * string(j) * (j < q - 1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:(q - 1)
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [15]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.8342752296337807e6
lmm.∇β = [11.772120067093992, -1925.6905435284602, -590.4811426628462, 2112.4417700624854, 2261.460483839501]
lmm.∇σ² = [911.8747620552729]
lmm.∇L = [24.41990131135614 24.011922772756076 16.56376848157203; 30.99925900310029 -59.40190266158887 46.58331413472227; 23.42470603064739 51.02946390987321 12.038870789780153]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [16]:
@assert abs(obj - (-2.8342752296337797e6)) < 1e-6
@assert norm(lmm.∇β - [11.772120067073923, -1925.690543528459, 
        -590.4811426628461, 2112.441770062488, 2261.4604838395016]) < 1e-6
@assert norm(lmm.∇L - [24.419901311377576 24.01192277273847 16.563768481574012; 
        30.99925900307824 -59.40190266157969 46.58331413470851; 
        23.424706030649975 51.029463909858194 12.038870789777398]) < 1e-6
@assert abs(lmm.∇σ²[1] - (911.8747620554551)) < 1e-6

### Efficiency

Test efficiency.

In [17]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.765 ms (0.00% GC)
  median time:      1.817 ms (0.00% GC)
  mean time:        1.849 ms (0.00% GC)
  maximum time:     3.571 ms (0.00% GC)
  --------------
  samples:          2700
  evals/sample:     1

My median run time is 2.122ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [18]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [19]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least sqaures solutions (ignoring intra-individual correlations)
\begin{eqnarray*}
\boldsymbol{\beta}^{(0)} &=& \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &=& \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{eqnarray*}
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 

Let $f = \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2 = \text{tr}(\sum_i (\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T)^T (\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T))$. Then

\begin{eqnarray*}
\text{d} f &=& \text{tr} (\sum_i - 2 \mathbf{Z}_i^T (\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T) \mathbf{Z}_i \text{d} \boldsymbol{\Sigma}) \\
\therefore \frac{\partial}{\partial \boldsymbol{\Sigma}} f &=& \sum_i - 2 \mathbf{Z}_i^T (\mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T) \mathbf{Z}_i \\
&& \rightarrow \sum_i \mathbf{Z}_i^T \mathbf{Z}_i \boldsymbol{\Sigma}^{(0)} \mathbf{Z}_i^T \mathbf{Z}_i = 
\sum_i \mathbf{Z}_i^T \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} \mathbf{Z}_i \\
&& \text{vec} (\sum_i \mathbf{Z}_i^T \mathbf{Z}_i \boldsymbol{\Sigma}^{(0)} \mathbf{Z}_i^T \mathbf{Z}_i) = 
\text{vec}(\sum_i \mathbf{Z}_i^T \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} \mathbf{Z}_i) \\
&& \sum_i (\mathbf{Z}_i^T \mathbf{Z}_i \otimes \mathbf{Z}_i^T \mathbf{Z}_i) \text{vec} (\boldsymbol{\Sigma}^{(0)}) = \sum_i (\mathbf{Z}_i^T \mathbf{r}_i^{(0)} \otimes \mathbf{Z}_i^T \mathbf{r}_i^{(0)}) \\
\therefore \text{vec} (\boldsymbol{\Sigma}^{(0)}) &=& [\sum_i (\mathbf{Z}_i^T \mathbf{Z}_i \otimes \mathbf{Z}_i^T \mathbf{Z}_i)]^{-1} [\sum_i (\mathbf{Z}_i^T \mathbf{r}_i^{(0)} \otimes \mathbf{Z}_i^T \mathbf{r}_i^{(0)})]
\end{eqnarray*} 

We implement this start point strategy in the function `init_ls()`.

In [20]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # TODO: fill m.β, m.σ², m.L by LS estimates
    # initialize arrays
    n = T(0)
    m.xtx .= T(0)
    m.xty .= T(0)
    m.σ²[1] = 0
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        n += size(obs.X, 1)
        m.xtx .+= obs.xtx
        m.xty .+= obs.xty
        m.σ²[1] += obs.yty
    end
    # compute m.β
    copy!(m.storage_pp, m.xtx)
    copy!(m.β, m.xty)
    LAPACK.potrf!('U', m.storage_pp)
    LAPACK.potrs!('U', m.storage_pp, m.β)
    # compute m.σ²
    m.σ²[1] += dot(m.β, BLAS.gemv!('N', T(1), m.xtx, m.β, T(-2), m.xty))
    m.σ²[1] /= n
    # compute m.L
    m.ztz2 .= T(0)
    m.storage_qq .= T(0)
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        kron!(m.storage_qq2, obs.ztz, obs.ztz)
        BLAS.axpy!(T(1), m.storage_qq2, m.ztz2)
        copy!(m.ztr, obs.zty)
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), m.ztr)
        BLAS.ger!(T(1), m.ztr, m.ztr, m.storage_qq)
    end
    m.ztr2 .= vec(m.storage_qq)
    LAPACK.potrf!('U', m.ztz2)
    LAPACK.potrs!('U', m.ztz2, m.ztr2)
    m.L .= reshape(m.ztr2, q, q)
    LAPACK.potrf!('L', m.L)
end

init_ls!

In [21]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.3529912271664576e6
lmm.β = [0.1281424806084756, 6.497945035096697, -3.502138774770135, 1.0026609224782135, 5.002697971721905]
lmm.σ² = [5.701709885959523]
lmm.L = [1.442089600524507 0.05523492575567489 0.04021508338255686; 0.03830200684866266 1.0546394793870693 0.06543803114908457; 0.027886674564416868 0.061034995187494806 1.0014753586943541]


### Correctness

Your start points should have a log-likelihood larger than -3.352991e6 (10 pts). The points you get are

In [22]:
# this is the points you get
(logl!(lmm) >  -3.353e6) * 10

10

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. You get full credit (10 pts) if the median run time is within 1ms.

In [23]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 
  memory estimate:  176 bytes
  allocs estimate:  4
  --------------
  minimum time:     208.764 μs (0.00% GC)
  median time:      217.270 μs (0.00% GC)
  mean time:        223.764 μs (0.00% GC)
  maximum time:     602.101 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [24]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathProgBase.jl

We define the NLP problem using the modelling tool MathProgBase.jl. Start-up code is given below. Modify if necessary to accomodate your own code.

In [25]:
"""
    fit!(m::LmmModel, solver = Ipopt.IpoptSolver(print_level = 5))

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m::LmmModel,
        solver = Ipopt.IpoptSolver(print_level = 5)
    )
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    optm = MathProgBase.NonlinearModel(solver)
    # set lower bounds and upper bounds of parameters
    # diagonal entries of Cholesky factor L should be >= 0
    lb   = fill(-Inf, npar)
    ub   = fill( Inf, npar)
    offset = p + 1
    for j in 1:q, i in j:q
        i == j && (lb[offset] = 0)
        offset += 1
    end
    # σ² should be >= 0
    lb[end] = 0
    MathProgBase.loadproblem!(optm, npar, 0, lb, ub, Float64[], Float64[], :Max, m)
    # starting point
    par0 = zeros(npar)
    modelpar_to_optimpar!(par0, m)
    MathProgBase.setwarmstart!(optm, par0)
    # optimize
    MathProgBase.optimize!(optm)
    optstat = MathProgBase.status(optm)
    optstat == :Optimal || @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    optimpar_to_modelpar!(m, MathProgBase.getsolution(optm))
    logl!(m, true)
    m
end

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MathProgBase.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in [:Grad])
            error("Unsupported feature $feat")
        end
    end
end

MathProgBase.features_available(m::LmmModel) = [:Grad]

function MathProgBase.eval_f(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MathProgBase.eval_grad_f(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

MathProgBase.eval_g(m::LmmModel, g, par) = nothing
MathProgBase.jac_structure(m::LmmModel) = Int[], Int[]
MathProgBase.eval_jac_g(m::LmmModel, J, par) = nothing

## Q7. (20 pts) Test drive

Now we can run our NLP solver to compute the MLE. For grading purpose, we first use the `:LD_LBFGS` (limited-memory BFGS) algorithm in NLopt.jl here.

In [26]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

@time fit!(lmm, NLopt.NLoptSolver(algorithm = :LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000));

println("objective value at solution: ", logl!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))

objective value at starting point: -3.3529912271664576e6

  0.846694 seconds (1.41 M allocations: 77.663 MiB, 2.20% gc time, 82.83% compilation time)
objective value at solution: -2.834264502363213e6

solution values:
lmm.β = [0.12386889603721703, 6.498337266205817, -3.5005130831538285, 1.001821941366638, 5.001953717754234]
lmm.σ² = [1.5023520948943991]
lmm.L * transpose(lmm.L) = [2.0685026437145684 0.053017368805560926 0.03271277214524364; 0.053017368805560926 1.1218137937601163 0.05601656427214081; 0.03271277214524364 0.05601656427214081 1.0120501850565866]
gradient @ solution:
lmm.∇β = [-0.012588368305703646, 0.022504964788772952, 0.033356548214975135, 0.018718182031133068, -0.0036046348391605143]
lmm.∇σ² = [-0.010259020848025102]
lmm.∇L = [0.0202890887444799 0.044878123309928006 0.002827388279535815; 0.059890364718108845 -0.013672271563402377 -0.019725544405109402; 0.0034260857332175172 -0.020555932336614 0.004478488243637489]
sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + abs2(nor

0.04861005677730496

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [27]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.83427e6
# gradient at solution should be small enough
@assert sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L))))) < 0.1

### Efficiency

My median run time is 132.5ms. You get 10 points if your median time is within 1s (=1000ms).

In [28]:
bm_bfgs = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm = :LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000))) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  12.20 KiB
  allocs estimate:  259
  --------------
  minimum time:     117.076 ms (0.00% GC)
  median time:      131.381 ms (0.00% GC)
  mean time:        131.836 ms (0.00% GC)
  maximum time:     150.239 ms (0.00% GC)
  --------------
  samples:          38
  evals/sample:     1

In [29]:
# this is the points you get
clamp(1 / (median(bm_bfgs).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods

Advantage of using a modelling tool such as MathProgBase.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solver works best. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/). 

In [30]:
# vector of solvers to compare
solvers = [
    # NLopt: gradient-based algorithms
    NLopt.NLoptSolver(algorithm = :LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12,
        xtol_rel = 1e-12, xtol_abs = 1e-12,
        maxeval=10000),
    NLopt.NLoptSolver(algorithm = :LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # NLopt: gradient-free algorithms
    NLopt.NLoptSolver(algorithm = :LN_BOBYQA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000),
    # Ipopt
    Ipopt.IpoptSolver(print_level = 0)
]
# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for (i, solver) in enumerate(solvers)
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [31]:
DataFrame(Runtime = runtime, Objective = objvals, Gradnorm = gradnrm)

,Runtime,Objective,Gradnorm
,Float64,Float64,Float64
1,0.119882,-2.83426e6,0.0486101
2,0.12911,-2.83426e6,0.00527842
3,0.0460048,-2.83539e6,31729.5
4,4.73216,-2.83426e6,6.73208e-6


The objective values (i.e. log-likelihood) were more or less the same across four different solvers with `:LN_BOBYQA` slightly smaller than the others. Meanwhile `:LN_BOBYQA` had the shortest run time, but also had the largest gradient norm, suggesting that there is some tradeoff here between performance and accuracy of the solution. This is also the case with `IpoptSolver`, which has the longest run time but the smallest graident norm. 

## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [32]:
method  = ["257", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [33]:
bm_257 = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm = :LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000))) setup = (init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.834264502360533e6

### lme4 

In [34]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.csv")
"""

┌ Warning: RCall.jl: Warning: package ‘lme4’ was built under R version 4.0.2
│ Loading required package: Matrix
│ Warning: package ‘Matrix’ was built under R version 4.0.2
└ @ RCall /Users/minsookim/.julia/packages/RCall/eRsxl/src/io.jl:160
┌ Warning: RCall.jl: Warning: package ‘readr’ was built under R version 4.0.2
└ @ RCall /Users/minsookim/.julia/packages/RCall/eRsxl/src/io.jl:160
┌ Warning: RCall.jl: Warning: package ‘magrittr’ was built under R version 4.0.2
└ @ RCall /Users/minsookim/.julia/packages/RCall/eRsxl/src/io.jl:160
┌ Warning: RCall.jl: 
│ ── Column specification ────────────────────────────────────────────────────────
│ cols(
│   ID = col_double(),
│   Y = col_double(),
│   X1 = col_double(),
│   X2 = col_double(),
│   X3 = col_double(),
│   X4 = col_double(),
│   Z1 = col_double(),
│   Z2 = col_double()
│ )
│ 
└ @ RCall /Users/minsookim/.julia/packages/RCall/eRsxl/src/io.jl:160


RObject{VecSxp}
# A tibble: 1,740,119 x 8
      ID       Y      X1     X2      X3      X4      Z1      Z2
   <dbl>   <dbl>   <dbl>  <dbl>   <dbl>   <dbl>   <dbl>   <dbl>
 1     1   2.26  -0.358   0.379 -0.107   0.784  -1.28   -1.96  
 2     1  10.5   -0.592  -0.540  1.14    1.46   -0.747   1.26  
 3     1 -13.6   -1.91    0.719  0.758  -0.865  -1.70    0.0890
 4     1   5.30  -0.106  -0.170  0.279   0.742  -0.0259  0.827 
 5     1  11.0   -0.0209 -1.80   1.72    0.761  -0.562  -1.47  
 6     1  -0.313  0.565   1.99  -0.437  -0.102   1.27    1.38  
 7     1  -0.688 -0.451   0.906  2.23   -0.386  -1.83    0.276 
 8     1  -4.50  -1.38   -0.527 -0.0893  0.210   1.19    1.49  
 9     1   8.48   1.20    0.148  1.19    0.172   2.22    1.11  
10     1  -4.39  -0.435   1.85  -0.146  -0.0681 -1.16    1.03  
# … with 1,740,109 more rows


In [35]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

RObject{RealSxp}
   user  system elapsed 
 90.551   5.107  95.715 


In [36]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl



In [37]:
testdata = CSV.File("lmm_data.csv", types = Dict(1 => String)) |> DataFrame!;

In [38]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

0.828005883

In [39]:
display(bm_mm)
mj

BenchmarkTools.Trial: 
  memory estimate:  879.31 MiB
  allocs estimate:  6110
  --------------
  minimum time:     786.158 ms (3.02% GC)
  median time:      828.006 ms (8.80% GC)
  mean time:        854.870 ms (9.97% GC)
  maximum time:     993.851 ms (21.01% GC)
  --------------
  samples:          6
  evals/sample:     1

|             |    Est. |     SE |        z |      p |   σ_ID |
|:----------- | -------:| ------:| --------:| ------:| ------:|
| (Intercept) |  0.1238 | 0.0455 |     2.72 | 0.0064 | 1.4383 |
| X1          |  6.4983 | 0.0009 |  6989.59 | <1e-99 |        |
| X2          | -3.5005 | 0.0009 | -3765.89 | <1e-99 |        |
| X3          |  1.0018 | 0.0009 |  1077.25 | <1e-99 |        |
| X4          |  5.0020 | 0.0009 |  5377.18 | <1e-99 |        |
| Z2          |         |        |          |        | 1.0060 |
| Z1          |         |        |          |        | 1.0592 |
| Residual    |  1.2257 |        |          |        |        |


### Summary

In [40]:
DataFrame(method = method, runtime = runtime, logl = loglike)

,method,runtime,logl
,String,Float64,Float64
1,257,0.131099,-2.83426e6
2,lme4,95.715,-2.83426e6
3,MixedModels.jl,0.828006,-2.83426e6


Overall, `:LD_MMA` algorithm showed the shortest runtime, while the objective values were more or less the same across different methods. 

## Q10. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by ~750 fold. 